**Importing Necessary Packages**

In [ ]:
############################################################################
## (C)Copyright 2021,2022 Hewlett Packard Enterprise Development LP
## Licensed under the Apache License, Version 2.0 (the "License"); you may
## not use this file except in compliance with the License. You may obtain
## a copy of the License at
##
##    http://www.apache.org/licenses/LICENSE-2.0
##
## Unless required by applicable law or agreed to in writing, software
## distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
## WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
## License for the specific language governing permissions and limitations
## under the License.
############################################################################

import os
import numpy as np
import csv
import logging
import tensorflow as tf

#from swarmlearning.tf import SwarmCallback Use Jupyter Notebook On Linux After Installing Wheel File

**Define Dataset Related Function and Parameters:**




In [ ]:
# Function TO Split Features and Labels Of A Dataset
def getXY(dataSet):
    np.random.shuffle(dataSet)
    length = np.size(dataSet,0)
    X = dataSet[0:length, :-1]
    y = dataSet[0:length, -1:]
    return X , y


# Constant Dataset Names for Training and Testing
testFileName = 'SB19_CCFDUBL_BAL_TEST_2C.csv'
fileNameList = [
    'SB19_CCFDUBL_BAL_TRAIN_2C.csv'
  , 'SB19_CCFDUBL_BAL_P1_2C.csv'
  , 'SB19_CCFDUBL_BAL_P2_2C.csv'
  , 'SB19_CCFDUBL_BAL_P3_2C.csv'
]

#Setting Batch Size and Epoch Parameters
part = 0
batchSize = 32
defaultMaxEpoch = 1000
defaultMinPeers = 2

**Main Model Training Function:**

Workflow Of Main Function:
1.   Set Paths for the dataDir,modelDir and scratchDir
2.   Load Training and Test Data
3.   Define the Model to be used to Train the Data
4.   Add the SwarmCallback from swarmlearning.tf
5.   Train the Model On Training Data
6.   Evaluate The Model Based On Testing Data
7.   Save the Model In the Path Mentioned





In [ ]:
def main():
  modelName = 'fraud-detection' #Change Value To Change Filename of New Model
  dataDir = os.getenv('DATA_DIR', '/platform/data') # Change to Path of Dataset
  modelDir = os.getenv('MODEL_DIR', '/platform/model') # Change to Path of Model
  scratchDir = os.getenv('SCRATCH_DIR', '/platform/scratch')  #Change to Path of Directory where New Model is Saved
  maxEpoch = int(os.getenv('MAX_EPOCHS', str(defaultMaxEpoch))) #Set maxEpochs 
  minPeers = int(os.getenv('MIN_PEERS', str(defaultMinPeers))) #Set minPeers

  print('***** Starting model =', modelName)


  # ================== Loading Test and Train Data =========================
  print('-' * 64)
  fname = fileNameList[part]
  trainFile = dataDir + '/' + fname
  print("Loading Train Dataset %s .." % trainFile)
  with open(trainFile, 'r') as f:
    # Removing Header From Train Data
    trainData = np.array(list(csv.reader(f, delimiter=","))[1:], dtype=float)
    print('size of training Data set : %s' % np.size(trainData,0))

  print('-' * 64)
  testFile = dataDir + '/' + testFileName
  print("Loading Test Dataset %s .." % testFile)
  with open(testFile, 'r') as f:
    # Removing Header From Test Data
    testData = np.array(list(csv.reader(f, delimiter=","))[1:], dtype=float)

  print('-' * 64)


  # ================== Define Model to Train and Evaluate =========================
  # Logistic Regression Model
  model = tf.keras.models.Sequential() #Defining TF Model

  model.add(tf.keras.layers.Dense(1, input_shape=(30,), activation='sigmoid',
    kernel_initializer='random_uniform', bias_initializer='zeros'))
  sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  print(model.summary())

  print('Starting training ...')
  x_train, y_train = getXY(trainData)
  x_test, y_test = getXY(testData)



  # ================== Adding Swarm Callback =========================
  swarmCallback = SwarmCallback(syncFrequency=128, minPeers=minPeers)

  

  # ================== Training the Model with Callback on Train Dataset =========================
  model.fit(
      x_train
    , y_train
    , batch_size=batchSize
    , epochs=maxEpoch
    , validation_data=(x_test, y_test)
    , shuffle=True
    , callbacks=[swarmCallback]
  )

  print('Training done!')

    # ================== Evaluate the Model on Test Dataset =========================
  scores = model.evaluate(x_test, y_test, verbose=1)
  print('***** Test loss:', scores[0])
  print('***** Test accuracy:', scores[1])


  # ================== Save the Model in the Scratch Directory =========================
  os.makedirs(scratchDir, exist_ok=True)
  model_path = os.path.join(scratchDir, modelName)
  model.save(model_path)
  print('Saved the trained model!')


if __name__ == '__main__':
    main()